#FIXED

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/BANGKIT-CAPSTONE/processed_dataset_lokasi.h5'
# Load and preprocess data
df = pd.read_hdf(file_path, 'df')

# Select relevant features
data_model = df[['Place_Name','City','Images','Price','Rating','Lat', 'Long','Category','Description','Hotel']]  # create a copy to avoid SettingWithCopyWarning

# Handle duplicates and missing data
data_model.drop_duplicates(inplace=True)
data_model.dropna(inplace=True)

# Function for recommendation based on city and budget
def recommend_places(city, budget, top_n=3):
    # Filter the data based on the input city and budget
    recommendations = data_model[(data_model['City'].str.contains(city, case=False, na=False)) & (data_model['Price'] <= budget)]

    # Calculate the absolute difference between the budget and the price, then sort the data based on this difference
    recommendations['abs_diff'] = abs(recommendations['Price'] - budget)
    recommendations.sort_values(by='abs_diff', ascending=True, inplace=True)

    recommendations = recommendations[['City', 'Price', 'Hotel']]
    # Return the top_n results
    return recommendations.head(top_n)

# Usage:
city = 'Jakarta'  # replace with your city
budget = 100000  # replace with your budget
hotel = 'Jakarta'
recommendations = recommend_places(city, budget)
print(recommendations)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       City   Price       Hotel
38  Jakarta  100000      Amaris
4   Jakarta   94000  Twin Plaza
42  Jakarta   70000        Win 


<ipython-input-2-22f86379e2f6>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations['abs_diff'] = abs(recommendations['Price'] - budget)
<ipython-input-2-22f86379e2f6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations.sort_values(by='abs_diff', ascending=True, inplace=True)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')
file_path = '/content/drive/My Drive/BANGKIT-CAPSTONE/dataset_lokasi(1).csv'

# Load and preprocess data
df = pd.read_csv(file_path)

# Select relevant features
data_model = df[['Place_Name','City','Images','Price','Rating','Lat', 'Long','Category','Description','Hotel']].copy()  # create a copy to avoid SettingWithCopyWarning

# Rescale the ratings to be between 1 and 5
data_model['Rating'] = data_model['Rating'].apply(lambda x: round(x/2) if x <= 10 else 5)

# Handle duplicates and missing data
data_model.drop_duplicates(inplace=True)
data_model.dropna(inplace=True)

# Split into features and target
X = df[['City','Price','Hotel']]
y = df['Rating']

# Convert City to one-hot encoding
X = pd.get_dummies(X, columns=['City', 'Hotel'])

# Convert y to categorical
y = tf.keras.utils.to_categorical(y-1, num_classes=5)

# Split into train, test, and validation sets (50% train, 20% validation, 30% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=(0.2/0.7), random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation='relu'))
model.add(Dense(5, activation='softmax'))  # 5 classes

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print('Train Accuracy: ', max(history.history['accuracy']))
print('Validation Accuracy: ', max(history.history['val_accuracy']))
print('Train Loss: ', min(history.history['loss']))
print('Validation Loss: ', min(history.history['val_loss']))

# Save the model and scaler for future use
data_model.to_hdf('processed_dataset_lokasi.h5', key='df', mode='w')
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save the data for future reference
data_model.to_csv('processed_dataset_lokasi.csv', index=False)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/100
7/7 [==============================] - 1s 55ms/step - loss: 1.0770 - accuracy: 0.7103 - val_loss: 1.0276 - val_accuracy: 0.9425
Epoch 2/100
7/7 [==============================] - 0s 11ms/step - loss: 0.7570 - accuracy: 0.8785 - val_loss: 0.8033 - val_accuracy: 0.9655
Epoch 3/100
7/7 [==============================] - 0s 16ms/step - loss: 0.5271 - accuracy: 0.9533 - val_loss: 0.6147 - val_accuracy: 0.9770
Epoch 4/100
7/7 [==============================] - 0s 18ms/step - loss: 0.3642 - accuracy: 0.9720 - val_loss: 0.4690 - val_accuracy: 0.9770
Epoch 5/100
7/7 [==============================] - 0s 16ms/step - loss: 0.2625 - accuracy: 0.9720 - val_loss: 0.3628 - val_accuracy: 0.9770
Epoch 6/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1951 - accuracy: 0.9766 - val_loss: 0.2906 - val_accuracy: 0.9770
Epoch 7/100
7/7 [==============